In [1]:
import sqlite3
import pandas as pd
from tqdm import tqdm_notebook
from functools import reduce

In [19]:
conn = sqlite3.connect("protein_training.db")

proteins_df = pd.read_sql("SELECT DISTINCT protein FROM protein_ngram", con=conn)
ngram_check = pd.read_sql("SELECT protein, gram_num FROM protein_ngram", con=conn)
amy_df = pd.read_sql("SELECT DISTINCT protein FROM amyloid", con=conn)

# Sanity check that all proteins are ones I plan on using
print(proteins_df[proteins_df['protein'].isin(amy_df['protein'])==False].shape[0])

conn.close()

0


In [18]:
proteins_df[proteins_df.duplicated()]

Empty DataFrame
Columns: [protein]
Index: []

In [20]:
ngram_check[ngram_check.duplicated()]

protein  gram_num
11301     R212B_HUMAN       109
22602     RBMS2_HUMAN       243
33903      RADX_HUMAN       752
45204     REG3A_HUMAN        43
56505      RAI1_HUMAN      1642
67806     REX1B_HUMAN       191
79107     RHF2B_HUMAN        79
90408     RHG31_HUMAN       805
101709    RASLC_HUMAN       205
113010    RBY1A_HUMAN        77
124311    RGPD8_HUMAN      1716
135612    REEP2_HUMAN       174
146913    RHBL1_HUMAN       183
158214    RGAP1_HUMAN       382
169515    RBGP1_HUMAN       417
180816    RGPD3_HUMAN       727
192117    RDH10_HUMAN       119
203418    RGPS2_HUMAN       366
214719    RHBL3_HUMAN       375
226020    RHG17_HUMAN       808
237321    RHG35_HUMAN       749
248622     RFX3_HUMAN       291
259923    RIMB1_HUMAN       221
271224    SMC1A_HUMAN       350
282525    SO1A2_HUMAN        22
293826    ST1C4_HUMAN       142
305127    SPON1_HUMAN       200
316428     SSR2_HUMAN       359
327729    STKL1_HUMAN       416
339030    SPT13_HUMAN       634
...               ...       ...
10973271  A16A1_HUMAN       498
10984572  ABCA8_HUMAN       820
10995873  ABCAD_HUMAN      1030
11007174  ABHEA_HUMAN         4
11018475   2B1B_HUMAN       163
11029776   AAA1_HUMAN       241
11041077   ARSH_HUMAN       442
11052378  AT131_HUMAN       903
11063679  CCYL1_HUMAN       160
11074980  AT11C_HUMAN        65
11086281  ATD2B_HUMAN       203
11097582  ARMD2_HUMAN       145
11108883   CCNF_HUMAN       498
11120184    ATM_HUMAN      2957
11131485  CCER1_HUMAN       262
11142786  AT2B2_HUMAN      1164
11154087   PEX6_HUMAN       271
11165388   NKRF_HUMAN       446
11176689    NPL_HUMAN       315
11187990  NLRC5_HUMAN       578
11199291  NU107_HUMAN       838
11210592   PTN5_HUMAN        15
11221893  PXDC2_HUMAN       397
11233194   PTX4_HUMAN        56
11244495  NDRG1_HUMAN       324
11255796   NET4_HUMAN       395
11267097   PGBM_HUMAN      1525
11278398  PTPRR_HUMAN       145
11289699  PXDNL_HUMAN       427
11301000   NEW1_YEAST      1023

[1000 rows x 2 columns]

In [ ]:
# Lets chunk this by protein name
chunk_size = 2000
chunk_list = list(range(0, proteins_df.shape[0],chunk_size))
chunk_pairs = [[chunk_list[i-1], chunk_list[i]] for i, x in enumerate(chunk_list) if i != 0]
chunk_pairs.append([chunk_pairs[-1][-1], None])
print(chunk_pairs[:5],"...",chunk_pairs[-5:])

In [3]:
proteins_df.shape[0]

20454

In [8]:
# Iterant 2, select subset, then join

conn = sqlite3.connect("protein_training.db")

for chunk in tqdm_notebook(chunk_pairs):
    start = chunk[0]
    end = chunk[1]
    
    if end != None:
        protein_chunk = proteins_df.iloc[start:end, :]
    else:
        protein_chunk = proteins_df.iloc[start:, :]
        
    prot_list = "("+", ".join(["'"+x+"'" for x in protein_chunk.protein.values.tolist()])+")"
    
    ngram_chunk = pd.read_sql(
        "SELECT \
            protein, gram_num, gram_1, gram_2, gram_3, gram_4, gram_5, \
            amyloid_1, amyloid_2, amyloid_3, amyloid_4, amyloid_5 \
        FROM protein_ngram WHERE protein IN "+prot_list, 
        con=conn
    )
    
    ngram_feat_chunk = pd.read_sql(
        "SELECT * FROM protein_ngram_features WHERE protein IN "+prot_list, 
        con=conn
    )
    
    ngram_sub_feat_chunk = pd.read_sql(
        "SELECT * FROM protein_subngram_features WHERE protein IN "+prot_list, 
        con=conn
    )
    
    frames = [ngram_chunk, ngram_feat_chunk, ngram_sub_feat_chunk]
    
    train_chunk = reduce(
        lambda left,right: pd.merge(
            left,right,on=['protein','gram_num'],
            how='inner'
        ),
        frames
    )
    
    protein_chunk = pd.read_sql(
        "SELECT * FROM protein_seq_features WHERE protein IN "+prot_list,
        con=conn
    )
    
    train_chunk = train_chunk.merge(protein_chunk, on=["protein"])

    train_chunk.to_sql("train_final", con=conn, index=False, if_exists="append")
    
conn.close()

# Takes 67 sec per iterant

In [9]:
# Sanity check
conn = sqlite3.connect("protein_training.db")

c = conn.cursor()

nrow_protein_ngram = [x[0] for x in c.execute("SELECT COUNT(*) FROM protein_ngram")][0]
nrow_train = [x[0] for x in c.execute("SELECT COUNT(*) FROM train_final")][0]

conn.close()

nrow_protein_ngram == nrow_train

False

In [10]:
nrow_protein_ngram

11319554

In [11]:
nrow_train

11325554

In [12]:
nrow_train - nrow_protein_ngram

6000

In [13]:
# Find the duplicates
# Sanity check
conn = sqlite3.connect("protein_training.db")

train_prot_gram = pd.read_sql("SELECT protein, gram_num from train_final", con=conn)

conn.close()

In [17]:
train_dup = train_prot_gram[train_prot_gram.duplicated(keep="first")]
train_dup_ind = train_dup.index.values.tolist()
print(len(train_dup_ind))

7000


In [48]:
# Check for null values in all the columns

conn = sqlite3.connect("protein_training.db")

c = conn.cursor()

train_columns = [x[1] for x in c.execute("PRAGMA table_info(train_final)")]

conn.close()

In [50]:
conn = sqlite3.connect("protein_training.db")

c = conn.cursor()

col_nulls = {}

for col in tqdm_notebook(train_columns):
    na_cells = [x[0] for x in c.execute(
        "SELECT COUNT(*) \
        FROM train_final \
            WHERE "+col+" IS NULL"
    )][0]
    
    col_nulls[col] = na_cells

conn.close()

In [51]:
col_nulls

{'protein': 0,
 'gram_num': 0,
 'gram_1': 0,
 'gram_2': 0,
 'gram_3': 0,
 'gram_4': 0,
 'gram_5': 0,
 'accession': 0,
 'amyloid_1': 0,
 'amyloid_2': 0,
 'amyloid_3': 0,
 'amyloid_4': 0,
 'amyloid_5': 0,
 'gram_frac': 0,
 'gram_mol_weight': 0,
 'gram_arom': 0,
 'gram_insta': 0,
 'gram_gravy': 0,
 'gram_isoel': 0,
 'gram_helix_perc': 0,
 'gram_turn_perc': 0,
 'gram_sheet_perc': 0,
 'gram_reduced_cys_num': 0,
 'gram_disulfide_num': 0,
 'gram_1_side_class': 1,
 'gram_1_side_polarity': 1,
 'gram_1_side_charge': 1,
 'gram_1_hydropathy_index': 1,
 'gram_1_mol_weight': 1,
 'gram_2_side_class': 1,
 'gram_2_side_polarity': 1,
 'gram_2_side_charge': 1,
 'gram_2_hydropathy_index': 1,
 'gram_2_mol_weight': 1,
 'gram_3_side_class': 1,
 'gram_3_side_polarity': 1,
 'gram_3_side_charge': 1,
 'gram_3_hydropathy_index': 1,
 'gram_3_mol_weight': 1,
 'gram_4_side_class': 1,
 'gram_4_side_polarity': 1,
 'gram_4_side_charge': 1,
 'gram_4_hydropathy_index': 1,
 'gram_4_mol_weight': 1,
 'gram_5_side_class': 1,